In [ ]:
import cv2 
import smart_attendance 
import os 
import numpy as np 

In [ ]:
import os
import cv2
import numpy as np
import smart_attendance

def load_known_faces(folder_path='known_faces'):
    known_encodings = []
    known_names = []

    for person_name in os.listdir(folder_path):
        person_folder = os.path.join(folder_path, person_name)
        if not os.path.isdir(person_folder):
            continue

        for img_name in os.listdir(person_folder):
            img_path = os.path.join(person_folder, img_name)

            img_bgr = cv2.imread(img_path)

            if img_bgr is None:
                print(f"❌ Failed to load: {img_path}")
                continue

            # Convert to grayscale
            gray_img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

            # Convert back to RGB from grayscale
            img_rgb = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2RGB)

            # Safety checks
            if img_rgb.dtype != np.uint8 or img_rgb.shape[2] != 3:
                print(f"❌ Skipping invalid image: {img_path}")
                continue

            try:
                encodings = smart_attendance.face_encodings(img_rgb)
                if encodings:
                    known_encodings.append(encodings[0])
                    known_names.append(person_name)
                    print(f"✅ Encoded: {img_path}")
                else:
                    print(f"⚠️ No face detected in: {img_path}")
            except Exception as e:
                print(f"❌ Error encoding {img_path}: {e}")

    return known_encodings, known_names


In [ ]:
def recognize_faces_with_keypress(known_face_encodings, known_face_names):
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("❌ Could not open webcam.")
        return

    print("🎥 Press 'q' in the video window to quit.")

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("⚠️ Failed to grab frame.")
                break

            small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
            rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

            face_locations = smart_attendance.face_locations(rgb_small_frame)
            face_encodings = smart_attendance.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                matches = smart_attendance.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"
                face_distances = smart_attendance.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]
                face_names.append(name)

            # Draw labels and boxes
            for (top, right, bottom, left), name in zip(face_locations, face_names):
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4

                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
                cv2.putText(frame, name, (left + 6, bottom - 6),
                            cv2.FONT_HERSHEY_DUPLEX, 0.9, (0, 0, 0), 1)

            cv2.imshow("Face Recognition (press 'q' to quit)", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                print("👋 Quitting...")
                break

    finally:
        cap.release()
        cv2.destroyAllWindows()
        print("✅ Webcam released.")


In [5]:
known_face_encodings, known_face_names = load_known_faces()

if known_face_encodings:
    recognize_faces_with_keypress(known_face_encodings, known_face_names)
else:
    print("❌ No valid face encodings found. Please check your images.")


✅ Encoded: known_faces\person1\img1.jpg
✅ Encoded: known_faces\person1\img2.jpg
✅ Encoded: known_faces\person2\img1.jpg
🎥 Press 'q' in the video window to quit.
👋 Quitting...
✅ Webcam released.
